Projektni zadatak 2 - Model za analizu sentimenata koji identifikuje emocije koje se prenose u datom tekstu



Korak 1 - Uvoz biblioteka i učitavanje ulaznih podataka

Prvo ćemo uvesti potrebne biblioteke za analizu teksta i analizu sentimenta, kao što su pandas za rukovanje podacima, nltk za obradu prirodnog jezika i SentimentlntensityAnalyzer za analizu sentimenta.

Kada je okruženje postavljeno, učitavamo skup podataka Amazon recenzija koristeći pd.read_csv(). Ovo će kreirati DataFrame objekat u Python-u koji možemo koristiti za analizu podataka. Sadržaj DataFrame-a prikazujemo koristeći df.

In [1]:
# Uvoz neophodnih biblioteka

import pandas as pd

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

# Učitavanje podataka
df = pd.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/amazon.csv')

df = df.head(100)

df = df.rename(columns={"Positive" : "Score"})

df

,reviewText,Score
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1
...,...,...
95,"I'll admit, I'm late to the game for getting t...",1
96,"I love angry birds, I have no issue with ad su...",0
97,I love the game and levels but I never have ti...,0
98,I see nothing wrong with this game but I busy ...,0


Korak 2 - Predprocesiranje teksta

Definišemo funkciju preprocess_text u kojoj prvo tokenizujemo tekst koristeći funkciju word_tokenize iz nltk biblioteke. Zatim uklanjamo stop-riječi iz tokenizovanog teksta koristeći modul stopwords iz nltk-a i na kraju lematizujemo filtrirane tokene pomoću modula WordNetLemmatizer (takođe iz nltk biblioteke). Pod lematizacijom riječi podrazumijevamo postupak svođenja riječi na osnovni oblik. Konačno, funkcija joins spaja lematizovane tokene nazad u string i vraća obrađeni tekst. 

In [2]:
# Definisemo preprocess_text funkciju

def preprocess_text(text):

    # Tokenizacija teksta
    tokens = word_tokenize(text.lower())
    
    # Uklanjanje stop-riječi
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    
    # Lematizacija tokena
    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Spajanje tokena nazad u string
    processed_text = ' '.join(lemmatized_tokens)

    return processed_text


# Pozivanje funkcije preprocess_text na DataFrame

df['reviewText'] = df['reviewText'].apply(preprocess_text)

df

,reviewText,Score
0,one best apps acording bunch people agree bomb...,1
1,pretty good version game free . lot different ...,1
2,really cool game . bunch level find golden egg...,1
3,"silly game frustrating , lot fun definitely re...",1
4,terrific game pad . hr fun . grandkids love . ...,1
...,...,...
95,"'ll admit , 'm late game getting wonderful gam...",1
96,"love angry bird , issue ad supported game . re...",0
97,love game level never time enjoy far get anoth...,0
98,see nothing wrong game busy n't think 's fun ....,0


Korak 3 - NLTK analizator sentimenta

Inicijalizujemo objekat klase SentimentIntensityAnalyzer iz nltk.sentiment.vader biblioteke. Inicijalizovani objekat analizira sentiment svakog teksta (recenzije) u DataFrame-u df. Za svaki tekst, analizator daje ocjene sentimenta kao rezultat. Rezultati analize se čuvaju rječniku res, gdje su ključevi redni brojevi recenzije (reda u tabeli), a vrijednosti su ocjene sentimenta. Na kraju, rezultati se pretvaraju u DataFrame vaders i prikazuju se ocjene za svaki sentiment.

Korišćenje biblioteke tqdm omogućava praćenje napretka analize tako što prikazuje traku napretka, odnosno koliko je redova obrađenjo u DataFrame-u.

In [3]:
from tqdm.notebook import tqdm

# Inicijalizacija NLTK sentiment analizatora
analyzer = SentimentIntensityAnalyzer()

res = {}

for i, row in tqdm(df.iterrows(), total = len(df)):
    text = row['reviewText']
    res[i] = analyzer.polarity_scores(text)

vaders = pd.DataFrame(res).T

vaders

  0%|          | 0/100 [00:00<?, ?it/s]

,neg,neu,pos,compound
0,0.146,0.548,0.306,0.5423
1,0.000,0.318,0.682,0.9325
2,0.000,0.417,0.583,0.8687
3,0.154,0.160,0.686,0.8402
4,0.000,0.286,0.714,0.9552
...,...,...,...,...
95,0.122,0.633,0.244,0.4389
96,0.088,0.559,0.353,0.8387
97,0.226,0.539,0.236,0.3182
98,0.273,0.590,0.137,-0.3711


Korak 4 - Analiza sentimenta teksta u DataFrame-u

Definišemo funkciju get_sentiment koja analizira sentiment teksta u koloni 'reviewText' svakog reda DataFrame-a. Na osnovu rezultata analize i vrijednosti u koloni 'Score', funkcija određuje da li je sentiment pozitivan, negativan ili neutralan. Rezultati se zatim dodaju u novu kolonu 'Sentiment' DataFrame-a df.

In [4]:
# Definisemo get_sentiment funkciju

def get_sentiment(row):

    scores = analyzer.polarity_scores(row['reviewText'])
    
    if scores['compound'] >= 0.05 and row['Score'] == 1:
        sentiment = 'positive'
    elif scores['compound'] <= -0.05 and row['Score'] == 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'

    return sentiment


# Pozivanje get_sentiment funkcije na DataFrame

df['Sentiment'] = df.apply(get_sentiment, axis=1)

df

,reviewText,Score,Sentiment
0,one best apps acording bunch people agree bomb...,1,positive
1,pretty good version game free . lot different ...,1,positive
2,really cool game . bunch level find golden egg...,1,positive
3,"silly game frustrating , lot fun definitely re...",1,positive
4,terrific game pad . hr fun . grandkids love . ...,1,positive
...,...,...,...
95,"'ll admit , 'm late game getting wonderful gam...",1,positive
96,"love angry bird , issue ad supported game . re...",0,neutral
97,love game level never time enjoy far get anoth...,0,neutral
98,see nothing wrong game busy n't think 's fun ....,0,negative


Korak 5 - Kasifikacija emocija



Učitavamo potrebne biblioteke, uključujući pipeline za analizu emocija. Zatim, preuzimamo prethodno obučeni model za analizu emocija. Koristimo taj model da izračunamo emocije za svaki red teksta u DataFrame-u. Rezultati se dodaju u DataFrame kao nove kolone koje sadrže oznake i ocjene emocija. Konačni rezultat je DataFrame koji sadrži informacije o emocijama koje su najviše zastupljene u svakoj recenziji.

In [5]:
from transformers import pipeline
import warnings

# Ne dozvoli FutureWarning poruke
warnings.simplefilter(action='ignore', category=FutureWarning)
    
# Preuzimanje modela za klasifikaciju emocija
model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

# Izračunavanje emocija za svaki red DataFrame-a
all_texts = df["reviewText"].values.tolist()
all_emotions = model(all_texts)

# Kreiranje novih kolona DataFrame-a
df["Emotion Label"] = [d["label"] for d in all_emotions]
df["Emotion Score"] = [d["score"] for d in all_emotions]

df

,reviewText,Score,Sentiment,Emotion Label,Emotion Score
0,one best apps acording bunch people agree bomb...,1,positive,neutral,0.475743
1,pretty good version game free . lot different ...,1,positive,joy,0.955907
2,really cool game . bunch level find golden egg...,1,positive,joy,0.955430
3,"silly game frustrating , lot fun definitely re...",1,positive,sadness,0.870134
4,terrific game pad . hr fun . grandkids love . ...,1,positive,joy,0.963085
...,...,...,...,...,...
95,"'ll admit , 'm late game getting wonderful gam...",1,positive,sadness,0.489019
96,"love angry bird , issue ad supported game . re...",0,neutral,anger,0.950331
97,love game level never time enjoy far get anoth...,0,neutral,neutral,0.341931
98,see nothing wrong game busy n't think 's fun ....,0,negative,sadness,0.383056


Korak 6 - Grafički prikaz emocija iz teksta

Koristimo plotly.express biblioteku kako bi grafički prikazali emocije pronađene u recenzijama na Amazonu. Koristimo histogram() funkciju za kreiranje dijagrama DataFrame-a df. Kolona 'Emotion Label' DataFrame-a df predstavlja x-osu dijagrama, a y-osa broj recenzija koje su povezane sa određenom emocijom. 

In [40]:
import plotly.express as px

# Pomocu histograma prikazujemo sve emocije u recenzijama na Amazonu
plot_title = "Emocije zastupljene u recenzijama na Amazonu"
fig = px.histogram(df, x="Emotion Label", template="plotly_dark",
                   title=plot_title, color="Emotion Label")
fig.update_layout(showlegend=False)
fig.show()